In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

%matplotlib inline

### Reading in and examining the heart attack cost disparities data

In [ ]:
ha_costs_df = pd.read_csv('../data/mmd_heart_attack_data.csv') 
ha_costs_df.head(2)

In [ ]:
tn_ha_costs = ha_costs_df.loc[ha_costs_df.state == 'TENNESSEE']
tn_ha_costs.shape

### Now examining the cancer data

In [ ]:
cancer_costs_df = pd.read_csv('../data/mmd_cancer_data.csv')
cancer_costs_df.head(2)

In [ ]:
tn_cancer_costs = cancer_costs_df.loc[cancer_costs_df.state == 'TENNESSEE']
tn_cancer_costs.shape

### Getting the income data and cleaning it a bit

In [ ]:
income_df = pd.read_csv('../data/irs_county_2016.csv')
income_df.head(2)

In [ ]:
tn_income = income_df.loc[income_df.STATE == 'TN']
tn_income.head(2)

In [ ]:
tn_income.shape

In [ ]:
tn_income = tn_income[['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']]

In [ ]:
tn_income.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']

### Week two coding tasks
#### Replacing coded values in the `income_bucket` column with descriptive text
- create a dictionary mapping codes to descriptions
- use `replace()` to update the df with text

In [ ]:
income_dict = {0:'Total', 1: 'Under $1', 2: 'Between 1 and $10,000', 3: 'Between 10,000 and $25,000',
              4: 'Between 25,000 and $50,000', 5: 'Between 50,000 and $75,000', 
               6: 'Between 75,000 and $100,000', 7: 'Between 100,000 and $200,000', 
               8:'$200,000 or more'}

In [ ]:
tn_income.income_bucket = tn_income.income_bucket.replace(income_dict)
tn_income.head(2)

#### Creating a new df that aggregates by county to get the totals for each county

In [ ]:
income_county_agg = tn_income.groupby('county').agg('sum').reset_index()
income_county_agg.head(2)

### Week 3 coding tasks

#### Create a barplot to show the total number of returns for each bucket
- first filter to use only the rows where the `county` is **Tennessee**

In [ ]:
statewide_tn_income = tn_income.loc[tn_income.county == 'Tennessee']
statewide_tn_income.head(2)

In [ ]:
plt.bar('income_bucket', 'return_count', data = statewide_tn_income)
plt.xticks(rotation = 90)
plt.title('Number of Returns by Income Bucket');

#### Create a new column: `avg_income`

In [ ]:
income_county_agg['avg_income'] = round(income_county_agg.total_inc_amt * 1000 / income_county_agg.returns_with_total_inc, 0)
income_county_agg.head(3)

#### Create a histogram to show distribution of average incomes in the counties

In [ ]:
income_county_agg = income_county_agg.loc[income_county_agg.county !='Tennessee']

In [ ]:
income_county_agg.avg_income.hist(bins = 12)
plt.title('Distribution of Average Incomes for TN Counties');

#### Create boxplots and swarmplots for the `analysis_value` in cancer and heart attack data to compare urban and rural counties

- cancer data

In [ ]:
plt.figure(figsize = (8, 6))

sns.boxplot(x = tn_cancer_costs.urban, y = tn_cancer_costs.analysis_value, order=["Urban", "Rural"])
plt.xlabel('')
plt.ylabel('')
plt.title('Cancer Analysis Value - Urban vs Rural');

In [ ]:
sns.swarmplot(x = tn_cancer_costs.urban, y = tn_cancer_costs.analysis_value, order=["Urban", "Rural"])
plt.xlabel('')
plt.ylabel('')
plt.title('Cancer Analysis Value - Urban vs Rural');

In [ ]:
sns.boxplot(x = tn_ha_costs.urban, y = tn_ha_costs.analysis_value, order=["Urban", "Rural"])
plt.xlabel('')
plt.ylabel('')
plt.title('Heart Attack Analysis Value - Urban vs Rural');

In [ ]:
sns.swarmplot(x = tn_ha_costs.urban, y = tn_ha_costs.analysis_value, order=["Urban", "Rural"])
plt.xlabel('')
plt.ylabel('')
plt.title('Heart Attack Analysis Value - Urban vs Rural');

#### Week 4 Coding Tasks
 - rename `analysis_value` columns in `tn_ha_costs` and `tn_cancer_costs`
 

In [ ]:
tn_ha_costs.head(2)

In [ ]:
tn_ha_costs = tn_ha_costs.rename(columns = {'analysis_value': 'ha_avg_cost'})
tn_ha_costs.head(2)

In [ ]:
tn_cancer_costs = tn_cancer_costs.rename(columns = {'analysis_value': 'cancer_avg_cost'})
tn_cancer_costs.head(2)

#### Create a new dataframe  `tn_df` 
- merge `county`, `urban`, and `ha_avg_cost` from `tn_ha_costs` with `county` and `cancer_avg_cost` from `tn_cancer_costs`
- check that `tn_df` contains all counties

In [ ]:
tn_cancer_costs.loc[~tn_cancer_costs.county.isin(tn_ha_costs.county)]

##### There is no data for Hancock, Moore, Pickett, and Van Buren counties in the heart attack data

In [ ]:
tn_cancer_cost_sub = tn_cancer_costs[['county', 'urban', 'cancer_avg_cost']]
tn_ha_cost_sub = tn_ha_costs[['county', 'ha_avg_cost']]

In [ ]:
tn_df = pd.merge(tn_ha_cost_sub, tn_cancer_cost_sub, on = 'county', how = 'inner')
tn_df.head(3)

In [ ]:
tn_df.county.nunique()

In [ ]:
plt.figure(figsize = (8,6))
plt.scatter(tn_df.cancer_avg_cost, tn_df.ha_avg_cost, marker = 'o')
plt.title('One Year Cancer Costs vs One Year Heart Attack Costs')
plt.xlabel('average annual cancer costs ($)')
plt.ylabel('average annual heart attack costs ($)');

- merge `avg_income` from `income_county_agg` with `tn_df` and save back to the `tn_df` variable  

- create `ha_cost_income_ratio` (`ha_avg_cost` / `avg_income`)  

- create `cancer_cost_income_ratio` (`cancer_avg_cost` / `avg_income`)

In [ ]:
income_county_agg.head(2)

In [ ]:
income_sub = income_county_agg[['county', 'avg_income']]
tn_df = pd.merge(tn_df, income_sub, on = 'county', how = 'inner')
tn_df.head(2)

In [ ]:
tn_df['ha_cost_income_ratio'] = tn_df.ha_avg_cost / tn_df.avg_income
tn_df['cancer_cost_income_ratio'] = tn_df.cancer_avg_cost / tn_df.avg_income
tn_df.head()

In [ ]:
counties = gpd.read_file('../data/county/tncounty.shp')
counties.head(2)

In [ ]:
counties.shape

#### Before you can merge the `counties` GeoDataFrame with `tn_df` you need to drop the word `County` to get a match
 - one way to do this is using string slicing
 - remember to make sure the counties GeoDataFrame is on the left so that the result is a GeoDataFrame


In [ ]:
tn_df['county2'] = tn_df.county.str[:-7]

In [ ]:
geo_tn = pd.merge(counties, tn_df, left_on = 'NAME', right_on = 'county2')
geo_tn.head(2)

#### Choropleth of `ha_cost_income_ratio`

In [ ]:
geo_tn[geo_tn.ha_cost_income_ratio.notnull()].shape

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
geo_tn[geo_tn.ha_cost_income_ratio.notnull()].plot(column = 'ha_cost_income_ratio', 
              cmap = 'BuPu', 
              edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('Heart Attack Cost to Income Ratio', fontsize = 14)
ax.axis('off');

#### Choropleth of `cancer_cost_income_ratio`

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
geo_tn.plot(column = 'cancer_cost_income_ratio', 
              cmap = 'BuPu', 
              edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('Cancer Cost to Income Ratio', fontsize = 14)
ax.axis('off');